---
title: push_wandb_artifact
author: Juma Shafara
date: '2024-11-19'
---

Uploads a dataset to Weights & Biases as an artifact.

In [ ]:

import os
import wandb
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

def push_wandb_artifact(file_path: str, 
                      job_type: str, 
                      description: str = 'no description', 
                      artifact_type: str = 'dataset',
                      project:str = "Risk Assessment",
                      ) -> None:
    """
    Uploads a dataset to Weights & Biases as an artifact.

    Args:
        file_path (str): Path to the file to upload.
        job_type (str): The type of job (e.g., "training", "evaluation").
        description (str, optional): Description of the artifact. Defaults to 'no description'.
        artifact_type (str, optional): Type of the artifact (e.g., "dataset"). Defaults to 'dataset'.
        project (str, optional): Name of the Weights & Biases project. Defaults to "Risk Assessment".

    Returns:
        None
    
    Raises:
        ValueError: If file_path or job_type are not provided.
        ConnectionError: If there is an issue connecting to Weights & Biases.
    """
    # Validate arguments
    if not file_path:
        raise ValueError("File path cannot be empty. Please provide a valid path to the data file.")
    if not job_type:
        raise ValueError("Job type cannot be empty. Please provide a valid job type.")

    # Initialize Weights & Biases connection
    try:
        wandb.login(key=os.environ.get("WANDB_API_KEY"))
        wandb.init(project=project, job_type=job_type)
    except wandb.errors.CommError as e:
        raise ConnectionError("Failed to connect to Weights & Biases. Check your API key and internet connection.") from e

    # Create and log artifact
    artifact = wandb.Artifact(
        name=f"{job_type}_{os.path.basename(file_path)}",
        type=artifact_type,
        description=description,
    )
    artifact.add_file(file_path)

    try:
        wandb.log_artifact(artifact)
    finally:
        wandb.finish()
